Report A: Iris

This report shows a comparison of different Deep Neural Networks models using the Iris dataset. The comparison is based on different parameters for the model. The parameters include the hidden layers of [5,5,5], [20,20,20], [100,100,100], the cross validation of 0 folds, 5 folds, and 10 folds, and the activation functions of rectifier, and tanh. The models are built using H2O through Python.


This imports the h2o library and connects with the h2o server. The server has to be running.

In [1]:
import h2o

# switch to java 8 "sdk use java 8.0.452-tem"
# java -jar h2o.jar
h2o.init(ip="localhost", port="54321")

Checking whether there is an H2O instance running at http://localhost:54321. connected.
Attempting to start a local H2O server...
  Java Version: openjdk version "21.0.6" 2025-01-21 LTS; OpenJDK Runtime Environment Temurin-21.0.6+7 (build 21.0.6+7-LTS); OpenJDK 64-Bit Server VM Temurin-21.0.6+7 (build 21.0.6+7-LTS, mixed mode, sharing)
  Starting server from /home/adam/GithubProjects/pndc-lab/.venv/lib/python3.12/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpikzz_vw2
  JVM stdout: /tmp/tmpikzz_vw2/h2o_adam_started_from_python.out
  JVM stderr: /tmp/tmpikzz_vw2/h2o_adam_started_from_python.err
  Server is running at http://127.0.0.1:54333
Connecting to H2O server at http://127.0.0.1:54333 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Europe/Warsaw
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.7
H2O_cluster_version_age:,2 months and 1 day
H2O_cluster_name:,H2O_from_python_adam_7vzxsu
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,4.820 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


Helper function to build and test deep neural network model

In [ ]:
from h2o.estimators.deeplearning import H2ODeepLearningEstimator
import math

# saving metric data
mean_error_and_time_metrics = []
mse_metrics = []
rmse_metrics = []

def run_experiment(dataset, features, target, hidden, nfolds, activation):
    train, test = dataset.split_frame(ratios=[0.8], seed=42)
    dnn = H2ODeepLearningEstimator(activation=activation, hidden=hidden, nfolds=nfolds, seed=42, reproducible=True)
    dnn.train(x=features, y=target, training_frame=train, validation_frame=test)
    #print(dnn.mean_per_class_error(xval=True))
    #print(dnn.run_time / 1000)
    # print(type(dnn))
    # print(dnn.mse())
    # print(dnn.rmse())
    print(dnn.show())
    mean_error_and_time_metrics.append([activation, "None" if nfolds == 0 else f"{nfolds}-folds", hidden, 
                    f"{round(dnn.mean_per_class_error(train=True) * 100, 3)}%", 
                    f"{round(dnn.mean_per_class_error(valid=True) * 100, 3)}%", 
                    f"{round(dnn.mean_per_class_error(xval=True) * 100,3)}%" if nfolds != 0 else "-", 
                    f"{round(dnn.run_time / 1000,4)} s"])
    mse_metrics.append([activation, "None" if nfolds == 0 else f"{nfolds}-folds", hidden, 
                    f"{round(dnn.mse(train=True), 4)}", 
                    f"{round(dnn.mse(valid=True),4)}",
                    f"{round(dnn.mse(xval=True), 4)}" if nfolds != 0 else "-"])
    rmse_metrics.append([activation, "None" if nfolds == 0 else f"{nfolds}-folds", hidden, 
                    f"{round(dnn.rmse(train=True),4)}", 
                    f"{round(dnn.rmse(valid=True),4)}",
                    f"{round(dnn.rmse(xval=True),4)}" if nfolds != 0 else "-"])

Loading in Iris dataset

In [3]:
iris = h2o.import_file(path="datasets/iris/iris.data", col_names=["sepal length", "sepal width", "petal length", "petal width", "class"])

features = ["sepal length", "sepal width", "petal length", "petal width"]
target = "class"

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


| Parameters          | Values    |
|---------------------|-----------|
| hidden layers       | [5,5,5]   |
| cross folds         | 0         |
| activation function | rectifier |



In [4]:
run_experiment(dataset=iris, features=features, target=target, hidden=[5,5,5], nfolds=0, activation="rectifier")
#print(metrics)

deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1748460557059_1


Status of Neuron Layers: predicting class, 3-class classification, multinomial distribution, CrossEntropy loss, 103 weights/biases, 6.6 KB, 1,210 training samples, mini-batch size 1
    layer    units    type       dropout    l1    l2    mean_rate              rate_rms                momentum    mean_weight           weight_rms          mean_bias           bias_rms
--  -------  -------  ---------  ---------  ----  ----  ---------------------  ----------------------  ----------  --------------------  ------------------  ------------------  -------------------
    1        4        Input      0.0
    2        5        Rectifier  0.0        0.0   0.0   0.0007944038618006744  0.00024688546545803547  0.0         -0.08850988820195198  0.4412868022918701  0.5685324992490279  0.21693813800811768
    3        5        Rectifier  0.0        0.0   0.0   0.0013222971803043037  0.0007636141963303089   0.0         -0.11439902618527413  0.4691739082336426  0.6901607554761903  0.15616893768310547
    4        5        Rectifier  0.0        0.0   0.0   0.0010398823127616198  0.0010711452923715115   0.0         -0.04753951147198677  0.3745073080062866  0.7778661912517877  0.24716413021087646
    5        3        Softmax               0.0   0.0   0.003021679512069871   0.004245582967996597    0.0         1.0397988309462864    1.5156335830688477  0.0040909605836033  0.20478194952011108

ModelMetricsMultinomial: deeplearning
** Reported on train data. **

MSE: 0.31824133263456167
RMSE: 0.5641288262751352
LogLoss: 0.971822392526423
Mean Per-Class Error: 0.3666666666666667
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
Iris-setosa    Iris-versicolor    Iris-virginica    Error     Rate
-------------  -----------------  ----------------  --------  --------
42             0                  0                 0         0 / 42
4              36                 0                 0.1       4 / 40
6              33                 0                 1         39 / 39
52             69                 0                 0.355372  43 / 121

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.644628
2    0.677686
3    1

ModelMetricsMultinomial: deeplearning
** Reported on validation data. **

MSE: 0.34708213864891646
RMSE: 0.5891367741441002
LogLoss: 1.041571921555573
Mean Per-Class Error: 0.3333333333333333
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
Iris-setosa    Iris-versicolor    Iris-virginica    Error    Rate
-------------  -----------------  ----------------  -------  -------
8              0                  0                 0        0 / 8
0              10                 0                 0        0 / 10
0              11                 0                 1        11 / 11
8              21                 0                 0.37931  11 / 29

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.62069
2    0.62069
3    1

Scoring History: 
    timestamp            duration    training_speed    epochs    iterations    samples    training_rmse    training_logloss    training_r2    training_classification_error    training_auc    training_pr_auc    validation_rmse    validation_logloss    validation_r2    valid

None


| Parameters          | Values     |
|---------------------|------------|
| hidden layers       | [20,20,20] |
| cross folds         | 0          |
| activation function | rectifier  |


In [5]:
run_experiment(dataset=iris, features=features, target=target, hidden=[20,20,20], nfolds=0, activation="rectifier")


deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1748460557059_2


Status of Neuron Layers: predicting class, 3-class classification, multinomial distribution, CrossEntropy loss, 1,003 weights/biases, 17.7 KB, 1,210 training samples, mini-batch size 1
    layer    units    type       dropout    l1    l2    mean_rate              rate_rms               momentum    mean_weight            weight_rms           mean_bias                bias_rms
--  -------  -------  ---------  ---------  ----  ----  ---------------------  ---------------------  ----------  ---------------------  -------------------  -----------------------  -------------------
    1        4        Input      0.0
    2        20       Rectifier  0.0        0.0   0.0   0.0014756249489437322  0.0007890632841736078  0.0         -0.089770201605279     0.29203498363494873  0.5179190492115147       0.044419065117836
    3        20       Rectifier  0.0        0.0   0.0   0.0024706080435134936  0.0019298950210213661  0.0         0.0036522174712445123  0.22414958477020264  1.0063216041539162       0.05537879467010498
    4        20       Rectifier  0.0        0.0   0.0   0.052739787744794736   0.21721118688583374    0.0         0.0023227406763180625  0.21478551626205444  0.9912608120154909       0.02587909996509552
    5        3        Softmax               0.0   0.0   0.056547343706673324   0.21663016080856323    0.0         -0.12961241992500921   1.2033863067626953   -0.00045966917322051715  0.02042357623577118

ModelMetricsMultinomial: deeplearning
** Reported on train data. **

MSE: 0.03633351866764879
RMSE: 0.19061353222593824
LogLoss: 0.13295344030449907
Mean Per-Class Error: 0.050427350427350436
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
Iris-setosa    Iris-versicolor    Iris-virginica    Error      Rate
-------------  -----------------  ----------------  ---------  -------
42             0                  0                 0          0 / 42
0              36                 4                 0.1        4 / 40
0              2                  37                0.0512821  2 / 39
42             38                 41                0.0495868  6 / 121

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.950413
2    1
3    1

ModelMetricsMultinomial: deeplearning
** Reported on validation data. **

MSE: 0.018836389936010698
RMSE: 0.1372457282978625
LogLoss: 0.09260855015299618
Mean Per-Class Error: 0.0
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
Iris-setosa    Iris-versicolor    Iris-virginica    Error    Rate
-------------  -----------------  ----------------  -------  ------
8              0                  0                 0        0 / 8
0              10                 0                 0        0 / 10
0              0                  11                0        0 / 11
8              10                 11                0        0 / 29

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    1
2    1
3    1

Scoring History: 
    timestamp            duration    training_speed    epochs    iterations    samples    training_rmse    training_logloss    training_r2    training_classification_error    training_auc    training_pr_auc    validation_rmse    validation_logloss    validation_

None


| Parameters          | Values        |
|---------------------|---------------|
| hidden layers       | [100,100,100] |
| cross folds         | 0             |
| activation function | rectifier     |

In [6]:
run_experiment(dataset=iris, features=features, target=target, hidden=[100,100,100], nfolds=0, activation="rectifier")


deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1748460557059_3


Status of Neuron Layers: predicting class, 3-class classification, multinomial distribution, CrossEntropy loss, 21,003 weights/biases, 254.9 KB, 1,210 training samples, mini-batch size 1
    layer    units    type       dropout    l1    l2    mean_rate              rate_rms              momentum    mean_weight             weight_rms           mean_bias               bias_rms
--  -------  -------  ---------  ---------  ----  ----  ---------------------  --------------------  ----------  ----------------------  -------------------  ----------------------  ---------------------
    1        4        Input      0.0
    2        100      Rectifier  0.0        0.0   0.0   0.0030958175582054535  0.002095324918627739  0.0         -0.012928298530605389   0.145907461643219    0.49455128892158406     0.008600987493991852
    3        100      Rectifier  0.0        0.0   0.0   0.006481550804275321   0.005188466981053352  0.0         -0.0024040898289666075  0.10131821036338806  0.9940328786680304      0.0062844641506671906
    4        100      Rectifier  0.0        0.0   0.0   0.06293354452793137    0.2248997688293457    0.0         0.0003199339206825243   0.10083338618278503  0.9992403330816156      0.002108040265738964
    5        3        Softmax               0.0   0.0   0.0831781054411355     0.2562659978866577    0.0         -0.013997034392086789   0.5385892391204834   -0.0005870869920749674  0.0009695386979728937

ModelMetricsMultinomial: deeplearning
** Reported on train data. **

MSE: 0.04643993823578863
RMSE: 0.21549927664794755
LogLoss: 0.1593548278312718
Mean Per-Class Error: 0.05833333333333333
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
Iris-setosa    Iris-versicolor    Iris-virginica    Error      Rate
-------------  -----------------  ----------------  ---------  -------
42             0                  0                 0          0 / 42
0              33                 7                 0.175      7 / 40
0              0                  39                0          0 / 39
42             33                 46                0.0578512  7 / 121

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.942149
2    1
3    1

ModelMetricsMultinomial: deeplearning
** Reported on validation data. **

MSE: 0.013704686833191952
RMSE: 0.11706701855429628
LogLoss: 0.05745311267719232
Mean Per-Class Error: 0.0
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
Iris-setosa    Iris-versicolor    Iris-virginica    Error    Rate
-------------  -----------------  ----------------  -------  ------
8              0                  0                 0        0 / 8
0              10                 0                 0        0 / 10
0              0                  11                0        0 / 11
8              10                 11                0        0 / 29

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    1
2    1
3    1

Scoring History: 
    timestamp            duration    training_speed    epochs    iterations    samples    training_rmse    training_logloss    training_r2    training_classification_error    training_auc    training_pr_auc    validation_rmse    validation_logloss    valida

None


| Parameters          | Values    |
|---------------------|-----------|
| hidden layers       | [5,5,5]   |
| cross folds         | 5         |
| activation function | rectifier |


In [7]:
run_experiment(dataset=iris, features=features, target=target, hidden=[5,5,5], nfolds=5, activation="rectifier")


deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1748460557059_4


Status of Neuron Layers: predicting class, 3-class classification, multinomial distribution, CrossEntropy loss, 103 weights/biases, 6.6 KB, 1,331 training samples, mini-batch size 1
    layer    units    type       dropout    l1    l2    mean_rate              rate_rms               momentum    mean_weight           weight_rms           mean_bias              bias_rms
--  -------  -------  ---------  ---------  ----  ----  ---------------------  ---------------------  ----------  --------------------  -------------------  ---------------------  -------------------
    1        4        Input      0.0
    2        5        Rectifier  0.0        0.0   0.0   0.0008151700865710154  0.0002717217430472374  0.0         -0.08767222110182046  0.44194722175598145  0.5721724534955372     0.22250217199325562
    3        5        Rectifier  0.0        0.0   0.0   0.0013343163137324154  0.0007876446470618248  0.0         -0.11190218612551689  0.47181499004364014  0.6911791056941394     0.16987454891204834
    4        5        Rectifier  0.0        0.0   0.0   0.0010627496225060894  0.0010980251245200634  0.0         -0.0494506349042058   0.3764185905456543   0.7753334695737217     0.25762999057769775
    5        3        Softmax               0.0   0.0   0.003069944679737091   0.0041672661900520325  0.0         1.038559877872467     1.5149564743041992   0.0024339261788463573  0.21384602785110474

ModelMetricsMultinomial: deeplearning
** Reported on train data. **

MSE: 0.2815305085118852
RMSE: 0.5305944859418398
LogLoss: 0.80256108563036
Mean Per-Class Error: 0.35833333333333334
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
Iris-setosa    Iris-versicolor    Iris-virginica    Error     Rate
-------------  -----------------  ----------------  --------  --------
42             0                  0                 0         0 / 42
3              37                 0                 0.075     3 / 40
3              36                 0                 1         39 / 39
48             73                 0                 0.347107  42 / 121

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.652893
2    0.752066
3    1

ModelMetricsMultinomial: deeplearning
** Reported on validation data. **

MSE: 0.30401836357619844
RMSE: 0.5513786027551291
LogLoss: 0.8497750580190118
Mean Per-Class Error: 0.3333333333333333
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
Iris-setosa    Iris-versicolor    Iris-virginica    Error    Rate
-------------  -----------------  ----------------  -------  -------
8              0                  0                 0        0 / 8
0              10                 0                 0        0 / 10
0              11                 0                 1        11 / 11
8              21                 0                 0.37931  11 / 29

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.62069
2    0.689655
3    1

ModelMetricsMultinomial: deeplearning
** Reported on cross-validation data. **

MSE: 0.44533954222061956
RMSE: 0.6673376523324752
LogLoss: 1.7268558490936599
Mean Per-Class Error: 0.5083333333333333
AUC table was not computed: it is either disabled (model parameter 'auc_t

None


| Parameters          | Values     |
|---------------------|------------|
| hidden layers       | [20,20,20] |
| cross folds         | 5          |
| activation function | rectifier  |


In [8]:
run_experiment(dataset=iris, features=features, target=target, hidden=[20,20,20], nfolds=5, activation="rectifier")

deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1748460557059_5


Status of Neuron Layers: predicting class, 3-class classification, multinomial distribution, CrossEntropy loss, 1,003 weights/biases, 17.7 KB, 1,331 training samples, mini-batch size 1
    layer    units    type       dropout    l1    l2    mean_rate              rate_rms               momentum    mean_weight           weight_rms           mean_bias               bias_rms
--  -------  -------  ---------  ---------  ----  ----  ---------------------  ---------------------  ----------  --------------------  -------------------  ----------------------  --------------------
    1        4        Input      0.0
    2        20       Rectifier  0.0        0.0   0.0   0.001709204835788114   0.0012822975404560566  0.0         -0.08963708444498479  0.29319679737091064  0.5201844931644302      0.046560704708099365
    3        20       Rectifier  0.0        0.0   0.0   0.0027486696831329027  0.00215261522680521    0.0         0.003814975968853105  0.22509104013442993  1.0070775625002857      0.05503067374229431
    4        20       Rectifier  0.0        0.0   0.0   0.05342542856971704    0.2168530821800232     0.0         0.00219297549512703   0.21521413326263428  0.9910402704118233      0.026537366211414337
    5        3        Softmax               0.0   0.0   0.057917632386185386   0.21596163511276245    0.0         -0.13044179544473689  1.2048778533935547   -0.0010842526607373413  0.021683722734451294

ModelMetricsMultinomial: deeplearning
** Reported on train data. **

MSE: 0.02815201301232738
RMSE: 0.1677856162259667
LogLoss: 0.1058989332392853
Mean Per-Class Error: 0.050427350427350436
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
Iris-setosa    Iris-versicolor    Iris-virginica    Error      Rate
-------------  -----------------  ----------------  ---------  -------
42             0                  0                 0          0 / 42
0              36                 4                 0.1        4 / 40
0              2                  37                0.0512821  2 / 39
42             38                 41                0.0495868  6 / 121

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.950413
2    1
3    1

ModelMetricsMultinomial: deeplearning
** Reported on validation data. **

MSE: 0.013748588389153469
RMSE: 0.11725437471221903
LogLoss: 0.07319759637948364
Mean Per-Class Error: 0.0
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
Iris-setosa    Iris-versicolor    Iris-virginica    Error    Rate
-------------  -----------------  ----------------  -------  ------
8              0                  0                 0        0 / 8
0              10                 0                 0        0 / 10
0              0                  11                0        0 / 11
8              10                 11                0        0 / 29

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    1
2    1
3    1

ModelMetricsMultinomial: deeplearning
** Reported on cross-validation data. **

MSE: 0.04882601814632504
RMSE: 0.22096610180370435
LogLoss: 0.17485193288397632
Mean Per-Class Error: 0.05918803418803418
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set

None


| Parameters          | Values        |
|---------------------|---------------|
| hidden layers       | [100,100,100] |
| cross folds         | 5             |
| activation function | rectifier     |

In [9]:
run_experiment(dataset=iris, features=features, target=target, hidden=[100,100,100], nfolds=5, activation="rectifier")

deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1748460557059_6


Status of Neuron Layers: predicting class, 3-class classification, multinomial distribution, CrossEntropy loss, 21,003 weights/biases, 254.9 KB, 1,331 training samples, mini-batch size 1
    layer    units    type       dropout    l1    l2    mean_rate              rate_rms              momentum    mean_weight             weight_rms           mean_bias               bias_rms
--  -------  -------  ---------  ---------  ----  ----  ---------------------  --------------------  ----------  ----------------------  -------------------  ----------------------  ---------------------
    1        4        Input      0.0
    2        100      Rectifier  0.0        0.0   0.0   0.0036640616679505912  0.003292911686003208  0.0         -0.013456125290031196   0.14929813146591187  0.49263857658502147     0.01170031726360321
    3        100      Rectifier  0.0        0.0   0.0   0.009116694100416498   0.00878814235329628   0.0         -0.0037580225790085024  0.10155430436134338  0.9909314258509246      0.00775204598903656
    4        100      Rectifier  0.0        0.0   0.0   0.0636435884086357     0.22352057695388794   0.0         0.0002623226516991053   0.10094237327575684  0.9991393435791416      0.0024024909362196922
    5        3        Softmax               0.0   0.0   0.08257527251155504    0.25007951259613037   0.0         -0.014407422728836536   0.538949728012085    -0.0007087150618146987  0.0010771150700747967

ModelMetricsMultinomial: deeplearning
** Reported on train data. **

MSE: 0.02019197108251633
RMSE: 0.1420984555951131
LogLoss: 0.0642632681573247
Mean Per-Class Error: 0.03333333333333333
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
Iris-setosa    Iris-versicolor    Iris-virginica    Error      Rate
-------------  -----------------  ----------------  ---------  -------
42             0                  0                 0          0 / 42
0              36                 4                 0.1        4 / 40
0              0                  39                0          0 / 39
42             36                 43                0.0330579  4 / 121

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.966942
2    1
3    1

ModelMetricsMultinomial: deeplearning
** Reported on validation data. **

MSE: 0.00529845615655419
RMSE: 0.0727904949602226
LogLoss: 0.023448386584170942
Mean Per-Class Error: 0.0
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
Iris-setosa    Iris-versicolor    Iris-virginica    Error    Rate
-------------  -----------------  ----------------  -------  ------
8              0                  0                 0        0 / 8
0              10                 0                 0        0 / 10
0              0                  11                0        0 / 11
8              10                 11                0        0 / 29

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    1
2    1
3    1

ModelMetricsMultinomial: deeplearning
** Reported on cross-validation data. **

MSE: 0.020672799636481554
RMSE: 0.1437803868282512
LogLoss: 0.07334566831216346
Mean Per-Class Error: 0.024999999999999998
AUC table was not computed: it is either disabled (model parameter 'auc_type

None


| Parameters          | Values    |
|---------------------|-----------|
| hidden layers       | [5,5,5]   |
| cross folds         | 10        |
| activation function | rectifier |


In [10]:
run_experiment(dataset=iris, features=features, target=target, hidden=[5,5,5], nfolds=10, activation="rectifier")


deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1748460557059_7


Status of Neuron Layers: predicting class, 3-class classification, multinomial distribution, CrossEntropy loss, 103 weights/biases, 6.6 KB, 1,331 training samples, mini-batch size 1
    layer    units    type       dropout    l1    l2    mean_rate              rate_rms               momentum    mean_weight           weight_rms           mean_bias              bias_rms
--  -------  -------  ---------  ---------  ----  ----  ---------------------  ---------------------  ----------  --------------------  -------------------  ---------------------  -------------------
    1        4        Input      0.0
    2        5        Rectifier  0.0        0.0   0.0   0.0008151700865710154  0.0002717217430472374  0.0         -0.08767222110182046  0.44194722175598145  0.5721724534955372     0.22250217199325562
    3        5        Rectifier  0.0        0.0   0.0   0.0013343163137324154  0.0007876446470618248  0.0         -0.11190218612551689  0.47181499004364014  0.6911791056941394     0.16987454891204834
    4        5        Rectifier  0.0        0.0   0.0   0.0010627496225060894  0.0010980251245200634  0.0         -0.0494506349042058   0.3764185905456543   0.7753334695737217     0.25762999057769775
    5        3        Softmax               0.0   0.0   0.003069944679737091   0.0041672661900520325  0.0         1.038559877872467     1.5149564743041992   0.0024339261788463573  0.21384602785110474

ModelMetricsMultinomial: deeplearning
** Reported on train data. **

MSE: 0.2815305085118852
RMSE: 0.5305944859418398
LogLoss: 0.80256108563036
Mean Per-Class Error: 0.35833333333333334
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
Iris-setosa    Iris-versicolor    Iris-virginica    Error     Rate
-------------  -----------------  ----------------  --------  --------
42             0                  0                 0         0 / 42
3              37                 0                 0.075     3 / 40
3              36                 0                 1         39 / 39
48             73                 0                 0.347107  42 / 121

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.652893
2    0.752066
3    1

ModelMetricsMultinomial: deeplearning
** Reported on validation data. **

MSE: 0.30401836357619844
RMSE: 0.5513786027551291
LogLoss: 0.8497750580190118
Mean Per-Class Error: 0.3333333333333333
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
Iris-setosa    Iris-versicolor    Iris-virginica    Error    Rate
-------------  -----------------  ----------------  -------  -------
8              0                  0                 0        0 / 8
0              10                 0                 0        0 / 10
0              11                 0                 1        11 / 11
8              21                 0                 0.37931  11 / 29

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.62069
2    0.689655
3    1

ModelMetricsMultinomial: deeplearning
** Reported on cross-validation data. **

MSE: 0.3620214673033224
RMSE: 0.6016821979278782
LogLoss: 1.1830400394803677
Mean Per-Class Error: 0.4166666666666667
AUC table was not computed: it is either disabled (model parameter 'auc_ty

None


| Parameters          | Values     |
|---------------------|------------|
| hidden layers       | [20,20,20] |
| cross folds         | 10         |
| activation function | rectifier  |

In [11]:
run_experiment(dataset=iris, features=features, target=target, hidden=[20,20,20], nfolds=10, activation="rectifier")

deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1748460557059_8


Status of Neuron Layers: predicting class, 3-class classification, multinomial distribution, CrossEntropy loss, 1,003 weights/biases, 17.7 KB, 1,331 training samples, mini-batch size 1
    layer    units    type       dropout    l1    l2    mean_rate              rate_rms               momentum    mean_weight           weight_rms           mean_bias               bias_rms
--  -------  -------  ---------  ---------  ----  ----  ---------------------  ---------------------  ----------  --------------------  -------------------  ----------------------  --------------------
    1        4        Input      0.0
    2        20       Rectifier  0.0        0.0   0.0   0.001709204835788114   0.0012822975404560566  0.0         -0.08963708444498479  0.29319679737091064  0.5201844931644302      0.046560704708099365
    3        20       Rectifier  0.0        0.0   0.0   0.0027486696831329027  0.00215261522680521    0.0         0.003814975968853105  0.22509104013442993  1.0070775625002857      0.05503067374229431
    4        20       Rectifier  0.0        0.0   0.0   0.05342542856971704    0.2168530821800232     0.0         0.00219297549512703   0.21521413326263428  0.9910402704118233      0.026537366211414337
    5        3        Softmax               0.0   0.0   0.057917632386185386   0.21596163511276245    0.0         -0.13044179544473689  1.2048778533935547   -0.0010842526607373413  0.021683722734451294

ModelMetricsMultinomial: deeplearning
** Reported on train data. **

MSE: 0.02815201301232738
RMSE: 0.1677856162259667
LogLoss: 0.1058989332392853
Mean Per-Class Error: 0.050427350427350436
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
Iris-setosa    Iris-versicolor    Iris-virginica    Error      Rate
-------------  -----------------  ----------------  ---------  -------
42             0                  0                 0          0 / 42
0              36                 4                 0.1        4 / 40
0              2                  37                0.0512821  2 / 39
42             38                 41                0.0495868  6 / 121

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.950413
2    1
3    1

ModelMetricsMultinomial: deeplearning
** Reported on validation data. **

MSE: 0.013748588389153469
RMSE: 0.11725437471221903
LogLoss: 0.07319759637948364
Mean Per-Class Error: 0.0
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
Iris-setosa    Iris-versicolor    Iris-virginica    Error    Rate
-------------  -----------------  ----------------  -------  ------
8              0                  0                 0        0 / 8
0              10                 0                 0        0 / 10
0              0                  11                0        0 / 11
8              10                 11                0        0 / 29

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    1
2    1
3    1

ModelMetricsMultinomial: deeplearning
** Reported on cross-validation data. **

MSE: 0.04259749431983666
RMSE: 0.2063916042862128
LogLoss: 0.15956536400232213
Mean Per-Class Error: 0.05064102564102565
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set 

None


| Parameters          | Values        |
|---------------------|---------------|
| hidden layers       | [100,100,100] |
| cross folds         | 10            |
| activation function | rectifier     |


In [12]:
run_experiment(dataset=iris, features=features, target=target, hidden=[100,100,100], nfolds=10, activation="rectifier")

deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1748460557059_9


Status of Neuron Layers: predicting class, 3-class classification, multinomial distribution, CrossEntropy loss, 21,003 weights/biases, 254.9 KB, 1,331 training samples, mini-batch size 1
    layer    units    type       dropout    l1    l2    mean_rate              rate_rms              momentum    mean_weight             weight_rms           mean_bias               bias_rms
--  -------  -------  ---------  ---------  ----  ----  ---------------------  --------------------  ----------  ----------------------  -------------------  ----------------------  ---------------------
    1        4        Input      0.0
    2        100      Rectifier  0.0        0.0   0.0   0.0036640616679505912  0.003292911686003208  0.0         -0.013456125290031196   0.14929813146591187  0.49263857658502147     0.01170031726360321
    3        100      Rectifier  0.0        0.0   0.0   0.009116694100416498   0.00878814235329628   0.0         -0.0037580225790085024  0.10155430436134338  0.9909314258509246      0.00775204598903656
    4        100      Rectifier  0.0        0.0   0.0   0.0636435884086357     0.22352057695388794   0.0         0.0002623226516991053   0.10094237327575684  0.9991393435791416      0.0024024909362196922
    5        3        Softmax               0.0   0.0   0.08257527251155504    0.25007951259613037   0.0         -0.014407422728836536   0.538949728012085    -0.0007087150618146987  0.0010771150700747967

ModelMetricsMultinomial: deeplearning
** Reported on train data. **

MSE: 0.02019197108251633
RMSE: 0.1420984555951131
LogLoss: 0.0642632681573247
Mean Per-Class Error: 0.03333333333333333
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
Iris-setosa    Iris-versicolor    Iris-virginica    Error      Rate
-------------  -----------------  ----------------  ---------  -------
42             0                  0                 0          0 / 42
0              36                 4                 0.1        4 / 40
0              0                  39                0          0 / 39
42             36                 43                0.0330579  4 / 121

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.966942
2    1
3    1

ModelMetricsMultinomial: deeplearning
** Reported on validation data. **

MSE: 0.00529845615655419
RMSE: 0.0727904949602226
LogLoss: 0.023448386584170942
Mean Per-Class Error: 0.0
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
Iris-setosa    Iris-versicolor    Iris-virginica    Error    Rate
-------------  -----------------  ----------------  -------  ------
8              0                  0                 0        0 / 8
0              10                 0                 0        0 / 10
0              0                  11                0        0 / 11
8              10                 11                0        0 / 29

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    1
2    1
3    1

ModelMetricsMultinomial: deeplearning
** Reported on cross-validation data. **

MSE: 0.009124567392148336
RMSE: 0.0955226014728888
LogLoss: 0.04265151500784462
Mean Per-Class Error: 0.008333333333333333
AUC table was not computed: it is either disabled (model parameter 'auc_type

None


| Parameters          | Values  |
|---------------------|---------|
| hidden layers       | [5,5,5] |
| cross folds         | 0       |
| activation function | tanh    |


In [13]:
run_experiment(dataset=iris, features=features, target=target, hidden=[5,5,5], nfolds=0, activation="tanh")


deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1748460557059_10


Status of Neuron Layers: predicting class, 3-class classification, multinomial distribution, CrossEntropy loss, 103 weights/biases, 6.6 KB, 1,210 training samples, mini-batch size 1
    layer    units    type     dropout    l1    l2    mean_rate              rate_rms               momentum    mean_weight            weight_rms           mean_bias              bias_rms
--  -------  -------  -------  ---------  ----  ----  ---------------------  ---------------------  ----------  ---------------------  -------------------  ---------------------  --------------------
    1        4        Input    0.0
    2        5        Tanh     0.0        0.0   0.0   0.00406237969873473    0.0019551897421479225  0.0         -0.12480813199654221   0.45762932300567627  0.09365131413637462    0.21387481689453125
    3        5        Tanh     0.0        0.0   0.0   0.0057125508319586515  0.0031769806519150734  0.0         -0.008434396358206869  0.507638692855835    -0.09621891889900129   0.028109274804592133
    4        5        Tanh     0.0        0.0   0.0   0.004793386792298407   0.004814131185412407   0.0         0.08727876946330071    0.4434596300125122   0.0033107839516819323  0.05063463747501373
    5        3        Softmax             0.0   0.0   0.012623202707618474   0.011916689574718475   0.0         1.0353447141125798     1.614211082458496    0.00714269063977165    0.054041847586631775

ModelMetricsMultinomial: deeplearning
** Reported on train data. **

MSE: 0.044428414735536735
RMSE: 0.21078048945653566
LogLoss: 0.18920389620117317
Mean Per-Class Error: 0.02542735042735043
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
Iris-setosa    Iris-versicolor    Iris-virginica    Error      Rate
-------------  -----------------  ----------------  ---------  -------
42             0                  0                 0          0 / 42
0              39                 1                 0.025      1 / 40
0              2                  37                0.0512821  2 / 39
42             41                 38                0.0247934  3 / 121

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.975207
2    1
3    1

ModelMetricsMultinomial: deeplearning
** Reported on validation data. **

MSE: 0.0423559158621431
RMSE: 0.20580552923122133
LogLoss: 0.18261832901488678
Mean Per-Class Error: 0.06060606060606061
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
Iris-setosa    Iris-versicolor    Iris-virginica    Error      Rate
-------------  -----------------  ----------------  ---------  ------
8              0                  0                 0          0 / 8
0              10                 0                 0          0 / 10
0              2                  9                 0.181818   2 / 11
8              12                 9                 0.0689655  2 / 29

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.931034
2    1
3    1

Scoring History: 
    timestamp            duration    training_speed    epochs    iterations    samples    training_rmse    training_logloss    training_r2    training_classification_error    training_auc    training_pr_auc    validation_rmse    validation_logloss    

None


| Parameters          | Values     |
|---------------------|------------|
| hidden layers       | [20,20,20] |
| cross folds         | 0          |
| activation function | tanh       |


In [14]:
run_experiment(dataset=iris, features=features, target=target, hidden=[20,20,20], nfolds=0, activation="tanh")


deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1748460557059_11


Status of Neuron Layers: predicting class, 3-class classification, multinomial distribution, CrossEntropy loss, 1,003 weights/biases, 17.7 KB, 1,210 training samples, mini-batch size 1
    layer    units    type     dropout    l1    l2    mean_rate              rate_rms               momentum    mean_weight           weight_rms           mean_bias              bias_rms
--  -------  -------  -------  ---------  ----  ----  ---------------------  ---------------------  ----------  --------------------  -------------------  ---------------------  --------------------
    1        4        Input    0.0
    2        20       Tanh     0.0        0.0   0.0   0.0031249675063008907  0.0029980745166540146  0.0         -0.09616810470615747  0.28624188899993896  0.015554607881593513   0.04609386622905731
    3        20       Tanh     0.0        0.0   0.0   0.0066445998499693815  0.007968999445438385   0.0         0.004673294586100382  0.22205626964569092  -0.01966232237799452   0.11928355693817139
    4        20       Tanh     0.0        0.0   0.0   0.010592800529266241   0.01285037025809288    0.0         0.008863859937191592  0.2176685929298401   -0.007360417791850801  0.051685601472854614
    5        3        Softmax             0.0   0.0   0.013192689574013153   0.007289540022611618   0.0         -0.1257822958752513   1.2127904891967773   0.023126714947734644   0.05391792953014374

ModelMetricsMultinomial: deeplearning
** Reported on train data. **

MSE: 0.026483073467623318
RMSE: 0.1627362082255308
LogLoss: 0.11017082681755958
Mean Per-Class Error: 0.033547008547008546
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
Iris-setosa    Iris-versicolor    Iris-virginica    Error      Rate
-------------  -----------------  ----------------  ---------  -------
42             0                  0                 0          0 / 42
0              37                 3                 0.075      3 / 40
0              1                  38                0.025641   1 / 39
42             38                 41                0.0330579  4 / 121

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.966942
2    1
3    1

ModelMetricsMultinomial: deeplearning
** Reported on validation data. **

MSE: 0.016163266355215446
RMSE: 0.12713483533326123
LogLoss: 0.08515931034652391
Mean Per-Class Error: 0.0
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
Iris-setosa    Iris-versicolor    Iris-virginica    Error    Rate
-------------  -----------------  ----------------  -------  ------
8              0                  0                 0        0 / 8
0              10                 0                 0        0 / 10
0              0                  11                0        0 / 11
8              10                 11                0        0 / 29

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    1
2    1
3    1

Scoring History: 
    timestamp            duration    training_speed    epochs    iterations    samples    training_rmse    training_logloss    training_r2    training_classification_error    training_auc    training_pr_auc    validation_rmse    validation_logloss    validation_r2    validation_classific

None


| Parameters          | Values  |
|---------------------|---------|
| hidden layers       | [100,100,100] |
| cross folds         | 0       |
| activation function | tanh    |


In [15]:
run_experiment(dataset=iris, features=features, target=target, hidden=[100,100,100], nfolds=0, activation="tanh")

deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1748460557059_12


Status of Neuron Layers: predicting class, 3-class classification, multinomial distribution, CrossEntropy loss, 21,003 weights/biases, 254.9 KB, 1,210 training samples, mini-batch size 1
    layer    units    type     dropout    l1    l2    mean_rate             rate_rms               momentum    mean_weight             weight_rms           mean_bias               bias_rms
--  -------  -------  -------  ---------  ----  ----  --------------------  ---------------------  ----------  ----------------------  -------------------  ----------------------  --------------------
    1        4        Input    0.0
    2        100      Tanh     0.0        0.0   0.0   0.004159690776286879  0.0034106802195310593  0.0         -0.012049963546523941   0.13802599906921387  -0.0011571556621969026  0.027797847986221313
    3        100      Tanh     0.0        0.0   0.0   0.027284601358207875  0.03090425580739975    0.0         0.00043973815760091383  0.10136187076568604  0.011239710749526715    0.06239859759807587
    4        100      Tanh     0.0        0.0   0.0   0.03163055266271113   0.04408727586269379    0.0         0.0011502977990577676   0.10110166668891907  0.009503141817289418    0.05240561068058014
    5        3        Softmax             0.0   0.0   0.03148678666523968   0.04656904935836792    0.0         -0.013380135345893602   0.5425794124603271   -0.0033977743297611365  0.03314696252346039

ModelMetricsMultinomial: deeplearning
** Reported on train data. **

MSE: 0.024951871225798217
RMSE: 0.15796161314002277
LogLoss: 0.08012923877157462
Mean Per-Class Error: 0.041666666666666664
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
Iris-setosa    Iris-versicolor    Iris-virginica    Error      Rate
-------------  -----------------  ----------------  ---------  -------
42             0                  0                 0          0 / 42
0              35                 5                 0.125      5 / 40
0              0                  39                0          0 / 39
42             35                 44                0.0413223  5 / 121

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.958678
2    1
3    1

ModelMetricsMultinomial: deeplearning
** Reported on validation data. **

MSE: 0.005450458303682505
RMSE: 0.07382721926012455
LogLoss: 0.03131809716030908
Mean Per-Class Error: 0.0
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
Iris-setosa    Iris-versicolor    Iris-virginica    Error    Rate
-------------  -----------------  ----------------  -------  ------
8              0                  0                 0        0 / 8
0              10                 0                 0        0 / 10
0              0                  11                0        0 / 11
8              10                 11                0        0 / 29

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    1
2    1
3    1

Scoring History: 
    timestamp            duration    training_speed    epochs    iterations    samples    training_rmse    training_logloss    training_r2    training_classification_error    training_auc    training_pr_auc    validation_rmse    validation_logloss    validation_r2    valid

None


| Parameters          | Values  |
|---------------------|---------|
| hidden layers       | [5,5,5] |
| cross folds         | 5       |
| activation function | tanh    |


In [16]:
run_experiment(dataset=iris, features=features, target=target, hidden=[5,5,5], nfolds=5, activation="tanh")

deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1748460557059_13


Status of Neuron Layers: predicting class, 3-class classification, multinomial distribution, CrossEntropy loss, 103 weights/biases, 6.6 KB, 1,331 training samples, mini-batch size 1
    layer    units    type     dropout    l1    l2    mean_rate             rate_rms               momentum    mean_weight            weight_rms          mean_bias              bias_rms
--  -------  -------  -------  ---------  ----  ----  --------------------  ---------------------  ----------  ---------------------  ------------------  ---------------------  --------------------
    1        4        Input    0.0
    2        5        Tanh     0.0        0.0   0.0   0.003956238244427368  0.0018821479752659798  0.0         -0.1274754888843745    0.4598900079727173  0.09600870239348465    0.21994853019714355
    3        5        Tanh     0.0        0.0   0.0   0.005797389270737767  0.0031278589740395546  0.0         -0.011970072146505117  0.5106394290924072  -0.09534477161760067   0.026895493268966675
    4        5        Tanh     0.0        0.0   0.0   0.00500065301777795   0.004962129518389702   0.0         0.08792268201708793    0.4476357698440552  5.836283769988793e-07  0.053533896803855896
    5        3        Softmax             0.0   0.0   0.014463946999361118  0.01558409258723259    0.0         1.0378643749902645     1.6198716163635254  0.006683148349845717   0.052901700139045715

ModelMetricsMultinomial: deeplearning
** Reported on train data. **

MSE: 0.039108472364359895
RMSE: 0.1977586214665745
LogLoss: 0.17096017995682966
Mean Per-Class Error: 0.033547008547008546
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
Iris-setosa    Iris-versicolor    Iris-virginica    Error      Rate
-------------  -----------------  ----------------  ---------  -------
42             0                  0                 0          0 / 42
0              37                 3                 0.075      3 / 40
0              1                  38                0.025641   1 / 39
42             38                 41                0.0330579  4 / 121

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.966942
2    1
3    1

ModelMetricsMultinomial: deeplearning
** Reported on validation data. **

MSE: 0.031605801425991
RMSE: 0.17778020538291378
LogLoss: 0.15286722414552198
Mean Per-Class Error: 0.030303030303030304
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
Iris-setosa    Iris-versicolor    Iris-virginica    Error      Rate
-------------  -----------------  ----------------  ---------  ------
8              0                  0                 0          0 / 8
0              10                 0                 0          0 / 10
0              1                  10                0.0909091  1 / 11
8              11                 10                0.0344828  1 / 29

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.965517
2    1
3    1

ModelMetricsMultinomial: deeplearning
** Reported on cross-validation data. **

MSE: 0.0798603401585091
RMSE: 0.2825957185778106
LogLoss: 0.2820172144275274
Mean Per-Class Error: 0.08397435897435897
AUC table was not computed: it is either disabled (model parameter 'auc_type' was

None


| Parameters          | Values     |
|---------------------|------------|
| hidden layers       | [20,20,20] |
| cross folds         | 5          |
| activation function | tanh       |


In [17]:
run_experiment(dataset=iris, features=features, target=target, hidden=[20,20,20], nfolds=5, activation="tanh")

deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1748460557059_14


Status of Neuron Layers: predicting class, 3-class classification, multinomial distribution, CrossEntropy loss, 1,003 weights/biases, 17.7 KB, 1,331 training samples, mini-batch size 1
    layer    units    type     dropout    l1    l2    mean_rate             rate_rms               momentum    mean_weight           weight_rms           mean_bias              bias_rms
--  -------  -------  -------  ---------  ----  ----  --------------------  ---------------------  ----------  --------------------  -------------------  ---------------------  --------------------
    1        4        Input    0.0
    2        20       Tanh     0.0        0.0   0.0   0.003194619263376808  0.0027629509568214417  0.0         -0.09628257676959037  0.2861133813858032   0.018624357113756353   0.05368533730506897
    3        20       Tanh     0.0        0.0   0.0   0.00679836213530507   0.006796954199671745   0.0         0.004610950427741045  0.22285544872283936  -0.025903431176020474  0.14610856771469116
    4        20       Tanh     0.0        0.0   0.0   0.016078666830726434  0.027247317135334015   0.0         0.008371028537512757  0.21925300359725952  -0.007915115850055485  0.062132418155670166
    5        3        Softmax             0.0   0.0   0.017161091999150814  0.013248637318611145   0.0         -0.12457126173345993  1.2194890975952148   0.028046195697887345   0.06631055474281311

ModelMetricsMultinomial: deeplearning
** Reported on train data. **

MSE: 0.01939605387695847
RMSE: 0.1392697162952466
LogLoss: 0.08206365265588877
Mean Per-Class Error: 0.033547008547008546
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
Iris-setosa    Iris-versicolor    Iris-virginica    Error      Rate
-------------  -----------------  ----------------  ---------  -------
42             0                  0                 0          0 / 42
0              37                 3                 0.075      3 / 40
0              1                  38                0.025641   1 / 39
42             38                 41                0.0330579  4 / 121

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.966942
2    1
3    1

ModelMetricsMultinomial: deeplearning
** Reported on validation data. **

MSE: 0.019049719982771274
RMSE: 0.13802072301930343
LogLoss: 0.07740656129628395
Mean Per-Class Error: 0.030303030303030304
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
Iris-setosa    Iris-versicolor    Iris-virginica    Error      Rate
-------------  -----------------  ----------------  ---------  ------
8              0                  0                 0          0 / 8
0              10                 0                 0          0 / 10
0              1                  10                0.0909091  1 / 11
8              11                 10                0.0344828  1 / 29

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.965517
2    1
3    1

ModelMetricsMultinomial: deeplearning
** Reported on cross-validation data. **

MSE: 0.042688405270279534
RMSE: 0.2066117258779848
LogLoss: 0.15708520382955357
Mean Per-Class Error: 0.05064102564102565
AUC table was not computed: it is either disabled (model parameter 'auc_typ

None


| Parameters          | Values        |
|---------------------|---------------|
| hidden layers       | [100,100,100] |
| cross folds         | 5             |
| activation function | tanh          |

In [18]:
run_experiment(dataset=iris, features=features, target=target, hidden=[100,100,100], nfolds=5, activation="tanh")


deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1748460557059_15


Status of Neuron Layers: predicting class, 3-class classification, multinomial distribution, CrossEntropy loss, 21,003 weights/biases, 254.9 KB, 1,210 training samples, mini-batch size 1
    layer    units    type     dropout    l1    l2    mean_rate             rate_rms               momentum    mean_weight             weight_rms           mean_bias               bias_rms
--  -------  -------  -------  ---------  ----  ----  --------------------  ---------------------  ----------  ----------------------  -------------------  ----------------------  --------------------
    1        4        Input    0.0
    2        100      Tanh     0.0        0.0   0.0   0.004032156371249584  0.0037082992494106293  0.0         -0.011992013747149031   0.13767528533935547  -0.0012748309559712512  0.031223848462104797
    3        100      Tanh     0.0        0.0   0.0   0.028817245344910772  0.03476572036743164    0.0         0.00042279174065561166  0.10136723518371582  0.013401418190007602    0.0725235641002655
    4        100      Tanh     0.0        0.0   0.0   0.03583685287609696   0.05325467884540558    0.0         0.0011592715465503715   0.10117065906524658  0.010926506065433503    0.05989593267440796
    5        3        Softmax             0.0   0.0   0.043673042762869346  0.06421789526939392    0.0         -0.013544082132478555   0.543464183807373    -0.004755613047322055   0.03998531401157379

ModelMetricsMultinomial: deeplearning
** Reported on train data. **

MSE: 0.01837003605453232
RMSE: 0.13553610609181715
LogLoss: 0.06400122711158798
Mean Per-Class Error: 0.042521367521367516
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
Iris-setosa    Iris-versicolor    Iris-virginica    Error      Rate
-------------  -----------------  ----------------  ---------  -------
42             0                  0                 0          0 / 42
0              39                 1                 0.025      1 / 40
0              4                  35                0.102564   4 / 39
42             43                 36                0.0413223  5 / 121

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.958678
2    1
3    1

ModelMetricsMultinomial: deeplearning
** Reported on validation data. **

MSE: 0.050216123581943765
RMSE: 0.22408954366936393
LogLoss: 0.14618962627286267
Mean Per-Class Error: 0.06060606060606061
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
Iris-setosa    Iris-versicolor    Iris-virginica    Error      Rate
-------------  -----------------  ----------------  ---------  ------
8              0                  0                 0          0 / 8
0              10                 0                 0          0 / 10
0              2                  9                 0.181818   2 / 11
8              12                 9                 0.0689655  2 / 29

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.931034
2    1
3    1

ModelMetricsMultinomial: deeplearning
** Reported on cross-validation data. **

MSE: 0.016407946308142844
RMSE: 0.12809350611230394
LogLoss: 0.06768358922598738
Mean Per-Class Error: 0.016666666666666666
AUC table was not computed: it is either disabled (mod

None


| Parameters          | Values  |
|---------------------|---------|
| hidden layers       | [5,5,5] |
| cross folds         | 10      |
| activation function | tanh    |

In [19]:
run_experiment(dataset=iris, features=features, target=target, hidden=[5,5,5], nfolds=10, activation="tanh")

deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1748460557059_16


Status of Neuron Layers: predicting class, 3-class classification, multinomial distribution, CrossEntropy loss, 103 weights/biases, 6.6 KB, 1,331 training samples, mini-batch size 1
    layer    units    type     dropout    l1    l2    mean_rate             rate_rms               momentum    mean_weight            weight_rms          mean_bias              bias_rms
--  -------  -------  -------  ---------  ----  ----  --------------------  ---------------------  ----------  ---------------------  ------------------  ---------------------  --------------------
    1        4        Input    0.0
    2        5        Tanh     0.0        0.0   0.0   0.003956238244427368  0.0018821479752659798  0.0         -0.1274754888843745    0.4598900079727173  0.09600870239348465    0.21994853019714355
    3        5        Tanh     0.0        0.0   0.0   0.005797389270737767  0.0031278589740395546  0.0         -0.011970072146505117  0.5106394290924072  -0.09534477161760067   0.026895493268966675
    4        5        Tanh     0.0        0.0   0.0   0.00500065301777795   0.004962129518389702   0.0         0.08792268201708793    0.4476357698440552  5.836283769988793e-07  0.053533896803855896
    5        3        Softmax             0.0   0.0   0.014463946999361118  0.01558409258723259    0.0         1.0378643749902645     1.6198716163635254  0.006683148349845717   0.052901700139045715

ModelMetricsMultinomial: deeplearning
** Reported on train data. **

MSE: 0.039108472364359895
RMSE: 0.1977586214665745
LogLoss: 0.17096017995682966
Mean Per-Class Error: 0.033547008547008546
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
Iris-setosa    Iris-versicolor    Iris-virginica    Error      Rate
-------------  -----------------  ----------------  ---------  -------
42             0                  0                 0          0 / 42
0              37                 3                 0.075      3 / 40
0              1                  38                0.025641   1 / 39
42             38                 41                0.0330579  4 / 121

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.966942
2    1
3    1

ModelMetricsMultinomial: deeplearning
** Reported on validation data. **

MSE: 0.031605801425991
RMSE: 0.17778020538291378
LogLoss: 0.15286722414552198
Mean Per-Class Error: 0.030303030303030304
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
Iris-setosa    Iris-versicolor    Iris-virginica    Error      Rate
-------------  -----------------  ----------------  ---------  ------
8              0                  0                 0          0 / 8
0              10                 0                 0          0 / 10
0              1                  10                0.0909091  1 / 11
8              11                 10                0.0344828  1 / 29

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.965517
2    1
3    1

ModelMetricsMultinomial: deeplearning
** Reported on cross-validation data. **

MSE: 0.06692445287430714
RMSE: 0.2586976089458639
LogLoss: 0.24876653197197593
Mean Per-Class Error: 0.08397435897435897
AUC table was not computed: it is either disabled (model parameter 'auc_type' w

None


| Parameters          | Values     |
|---------------------|------------|
| hidden layers       | [20,20,20] |
| cross folds         | 10         |
| activation function | tanh       |


In [20]:
run_experiment(dataset=iris, features=features, target=target, hidden=[20,20,20], nfolds=10, activation="tanh")

deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1748460557059_17


Status of Neuron Layers: predicting class, 3-class classification, multinomial distribution, CrossEntropy loss, 1,003 weights/biases, 17.7 KB, 1,331 training samples, mini-batch size 1
    layer    units    type     dropout    l1    l2    mean_rate             rate_rms               momentum    mean_weight           weight_rms           mean_bias              bias_rms
--  -------  -------  -------  ---------  ----  ----  --------------------  ---------------------  ----------  --------------------  -------------------  ---------------------  --------------------
    1        4        Input    0.0
    2        20       Tanh     0.0        0.0   0.0   0.003194619263376808  0.0027629509568214417  0.0         -0.09628257676959037  0.2861133813858032   0.018624357113756353   0.05368533730506897
    3        20       Tanh     0.0        0.0   0.0   0.00679836213530507   0.006796954199671745   0.0         0.004610950427741045  0.22285544872283936  -0.025903431176020474  0.14610856771469116
    4        20       Tanh     0.0        0.0   0.0   0.016078666830726434  0.027247317135334015   0.0         0.008371028537512757  0.21925300359725952  -0.007915115850055485  0.062132418155670166
    5        3        Softmax             0.0   0.0   0.017161091999150814  0.013248637318611145   0.0         -0.12457126173345993  1.2194890975952148   0.028046195697887345   0.06631055474281311

ModelMetricsMultinomial: deeplearning
** Reported on train data. **

MSE: 0.01939605387695847
RMSE: 0.1392697162952466
LogLoss: 0.08206365265588877
Mean Per-Class Error: 0.033547008547008546
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
Iris-setosa    Iris-versicolor    Iris-virginica    Error      Rate
-------------  -----------------  ----------------  ---------  -------
42             0                  0                 0          0 / 42
0              37                 3                 0.075      3 / 40
0              1                  38                0.025641   1 / 39
42             38                 41                0.0330579  4 / 121

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.966942
2    1
3    1

ModelMetricsMultinomial: deeplearning
** Reported on validation data. **

MSE: 0.019049719982771274
RMSE: 0.13802072301930343
LogLoss: 0.07740656129628395
Mean Per-Class Error: 0.030303030303030304
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
Iris-setosa    Iris-versicolor    Iris-virginica    Error      Rate
-------------  -----------------  ----------------  ---------  ------
8              0                  0                 0          0 / 8
0              10                 0                 0          0 / 10
0              1                  10                0.0909091  1 / 11
8              11                 10                0.0344828  1 / 29

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.965517
2    1
3    1

ModelMetricsMultinomial: deeplearning
** Reported on cross-validation data. **

MSE: 0.033758508973470464
RMSE: 0.18373488774174182
LogLoss: 0.128935438129024
Mean Per-Class Error: 0.05064102564102565
AUC table was not computed: it is either disabled (model parameter 'auc_type

None


| Parameters          | Values        |
|---------------------|---------------|
| hidden layers       | [100,100,100] |
| cross folds         | 10            |
| activation function | tanh          |


In [21]:
run_experiment(dataset=iris, features=features, target=target, hidden=[100,100,100], nfolds=10, activation="tanh")


deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1748460557059_18


Status of Neuron Layers: predicting class, 3-class classification, multinomial distribution, CrossEntropy loss, 21,003 weights/biases, 254.9 KB, 1,331 training samples, mini-batch size 1
    layer    units    type     dropout    l1    l2    mean_rate              rate_rms               momentum    mean_weight            weight_rms           mean_bias              bias_rms
--  -------  -------  -------  ---------  ----  ----  ---------------------  ---------------------  ----------  ---------------------  -------------------  ---------------------  -------------------
    1        4        Input    0.0
    2        100      Tanh     0.0        0.0   0.0   0.0040676253630954305  0.0038918424397706985  0.0         -0.011938645499758422  0.13801807165145874  -0.001222273853346009  0.03120899200439453
    3        100      Tanh     0.0        0.0   0.0   0.029967011399345937   0.037132903933525085   0.0         0.0004347789631246087  0.101430743932724    0.01328770500550237    0.0713283121585846
    4        100      Tanh     0.0        0.0   0.0   0.03696650638135616    0.05561606585979462    0.0         0.0011761647203831671  0.1012389063835144   0.010737009762593347   0.05850796401500702
    5        3        Softmax             0.0   0.0   0.0405731340094159     0.06273853778839111    0.0         -0.013618433151083688  0.5439467430114746   -0.005192262415338892  0.03812277317047119

ModelMetricsMultinomial: deeplearning
** Reported on train data. **

MSE: 0.01518257810410201
RMSE: 0.12321760468416033
LogLoss: 0.05127493722056702
Mean Per-Class Error: 0.016666666666666666
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
Iris-setosa    Iris-versicolor    Iris-virginica    Error      Rate
-------------  -----------------  ----------------  ---------  -------
42             0                  0                 0          0 / 42
0              38                 2                 0.05       2 / 40
0              0                  39                0          0 / 39
42             38                 41                0.0165289  2 / 121

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.983471
2    1
3    1

ModelMetricsMultinomial: deeplearning
** Reported on validation data. **

MSE: 0.013553166759757785
RMSE: 0.11641806887145047
LogLoss: 0.042643846625508575
Mean Per-Class Error: 0.030303030303030304
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
Iris-setosa    Iris-versicolor    Iris-virginica    Error      Rate
-------------  -----------------  ----------------  ---------  ------
8              0                  0                 0          0 / 8
0              10                 0                 0          0 / 10
0              1                  10                0.0909091  1 / 11
8              11                 10                0.0344828  1 / 29

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.965517
2    1
3    1

ModelMetricsMultinomial: deeplearning
** Reported on cross-validation data. **

MSE: 0.008637249577460903
RMSE: 0.0929368042137285
LogLoss: 0.037902879548781895
Mean Per-Class Error: 0.016666666666666666
AUC table was not computed: it is either disabled (model par

None


In [ ]:
from tabulate import tabulate
from IPython.display import Markdown, display, HTML
headers = ["Activation Function", "Cross Validation", "Hidden Layers", "Training Error %", "Validation Error %", "Cross Validation Mean Error %", "Training Time"]
#print(metrics)
#print(tabulate(mean_error_metrics, headers=headers, tablefmt="github"))
display(Markdown(tabulate(mean_error_and_time_metrics, headers=headers, tablefmt="github")))

| Activation Function   | Cross Validation   | Hidden Layers   | Training Error %   | Validation Error %   | Cross Validation Mean Error %   | Training Time   |
|-----------------------|--------------------|-----------------|--------------------|----------------------|---------------------------------|-----------------|
| rectifier             | None               | [5, 5, 5]       | 36.667%            | 33.333%              | -                               | 0.501 s         |
| rectifier             | None               | [20, 20, 20]    | 5.043%             | 0.0%                 | -                               | 0.449 s         |
| rectifier             | None               | [100, 100, 100] | 5.833%             | 0.0%                 | -                               | 0.931 s         |
| rectifier             | 5-folds            | [5, 5, 5]       | 35.833%            | 33.333%              | 50.833%                         | 0.061 s         |
| rectifier             | 5-folds            | [20, 20, 20]    | 5.043%             | 0.0%                 | 5.919%                          | 0.084 s         |
| rectifier             | 5-folds            | [100, 100, 100] | 3.333%             | 0.0%                 | 2.5%                            | 0.568 s         |
| rectifier             | 10-folds           | [5, 5, 5]       | 35.833%            | 33.333%              | 41.667%                         | 0.045 s         |
| rectifier             | 10-folds           | [20, 20, 20]    | 5.043%             | 0.0%                 | 5.064%                          | 0.069 s         |
| rectifier             | 10-folds           | [100, 100, 100] | 3.333%             | 0.0%                 | 0.833%                          | 0.536 s         |
| tanh                  | None               | [5, 5, 5]       | 2.543%             | 6.061%               | -                               | 0.038 s         |
| tanh                  | None               | [20, 20, 20]    | 3.355%             | 0.0%                 | -                               | 0.058 s         |
| tanh                  | None               | [100, 100, 100] | 4.167%             | 0.0%                 | -                               | 0.653 s         |
| tanh                  | 5-folds            | [5, 5, 5]       | 3.355%             | 3.03%                | 8.397%                          | 0.025 s         |
| tanh                  | 5-folds            | [20, 20, 20]    | 3.355%             | 3.03%                | 5.064%                          | 0.057 s         |
| tanh                  | 5-folds            | [100, 100, 100] | 4.252%             | 6.061%               | 1.667%                          | 0.59 s          |
| tanh                  | 10-folds           | [5, 5, 5]       | 3.355%             | 3.03%                | 8.397%                          | 0.025 s         |
| tanh                  | 10-folds           | [20, 20, 20]    | 3.355%             | 3.03%                | 5.064%                          | 0.053 s         |
| tanh                  | 10-folds           | [100, 100, 100] | 1.667%             | 3.03%                | 1.667%                          | 0.854 s         |

The first thing to notice from the results show that when using the Rectifier activation function, using the hidden layers of [20,20,20] and [100,100,100] greatly decreases the training error. The different between the error values for those hidden layers are very close but the training time for the hidden layer [100,100,100] is greatly increased. In terms of 

In [23]:
headers = ["Activation Function", "Cross Validation", "Hidden Layers", "Training Mean Square Error", "Validation Mean Square Error", "Cross Validation Mean Square Error"]

display(Markdown(tabulate(mse_metrics, headers=headers, tablefmt="github")))

| Activation Function   | Cross Validation   | Hidden Layers   |   Training Mean Square Error |   Validation Mean Square Error | Cross Validation Mean Square Error   |
|-----------------------|--------------------|-----------------|------------------------------|--------------------------------|--------------------------------------|
| rectifier             | None               | [5, 5, 5]       |                       0.3182 |                         0.3471 | -                                    |
| rectifier             | None               | [20, 20, 20]    |                       0.0363 |                         0.0188 | -                                    |
| rectifier             | None               | [100, 100, 100] |                       0.0464 |                         0.0137 | -                                    |
| rectifier             | 5-folds            | [5, 5, 5]       |                       0.2815 |                         0.304  | 0.4453                               |
| rectifier             | 5-folds            | [20, 20, 20]    |                       0.0282 |                         0.0137 | 0.0488                               |
| rectifier             | 5-folds            | [100, 100, 100] |                       0.0202 |                         0.0053 | 0.0207                               |
| rectifier             | 10-folds           | [5, 5, 5]       |                       0.2815 |                         0.304  | 0.362                                |
| rectifier             | 10-folds           | [20, 20, 20]    |                       0.0282 |                         0.0137 | 0.0426                               |
| rectifier             | 10-folds           | [100, 100, 100] |                       0.0202 |                         0.0053 | 0.0091                               |
| tanh                  | None               | [5, 5, 5]       |                       0.0444 |                         0.0424 | -                                    |
| tanh                  | None               | [20, 20, 20]    |                       0.0265 |                         0.0162 | -                                    |
| tanh                  | None               | [100, 100, 100] |                       0.025  |                         0.0055 | -                                    |
| tanh                  | 5-folds            | [5, 5, 5]       |                       0.0391 |                         0.0316 | 0.0799                               |
| tanh                  | 5-folds            | [20, 20, 20]    |                       0.0194 |                         0.019  | 0.0427                               |
| tanh                  | 5-folds            | [100, 100, 100] |                       0.0184 |                         0.0502 | 0.0164                               |
| tanh                  | 10-folds           | [5, 5, 5]       |                       0.0391 |                         0.0316 | 0.0669                               |
| tanh                  | 10-folds           | [20, 20, 20]    |                       0.0194 |                         0.019  | 0.0338                               |
| tanh                  | 10-folds           | [100, 100, 100] |                       0.0152 |                         0.0136 | 0.0086                               |

In [24]:
headers = ["Activation Function", "Cross Validation", "Hidden Layers", "Training Root Mean Square Error", "Validation Root Mean Square Error", "Cross Validation Root Mean Square Error"]

display(Markdown(tabulate(rmse_metrics, headers=headers, tablefmt="github")))

| Activation Function   | Cross Validation   | Hidden Layers   |   Training Root Mean Square Error |   Validation Root Mean Square Error | Cross Validation Root Mean Square Error   |
|-----------------------|--------------------|-----------------|-----------------------------------|-------------------------------------|-------------------------------------------|
| rectifier             | None               | [5, 5, 5]       |                            0.5641 |                              0.5891 | -                                         |
| rectifier             | None               | [20, 20, 20]    |                            0.1906 |                              0.1372 | -                                         |
| rectifier             | None               | [100, 100, 100] |                            0.2155 |                              0.1171 | -                                         |
| rectifier             | 5-folds            | [5, 5, 5]       |                            0.5306 |                              0.5514 | 0.6673                                    |
| rectifier             | 5-folds            | [20, 20, 20]    |                            0.1678 |                              0.1173 | 0.221                                     |
| rectifier             | 5-folds            | [100, 100, 100] |                            0.1421 |                              0.0728 | 0.1438                                    |
| rectifier             | 10-folds           | [5, 5, 5]       |                            0.5306 |                              0.5514 | 0.6017                                    |
| rectifier             | 10-folds           | [20, 20, 20]    |                            0.1678 |                              0.1173 | 0.2064                                    |
| rectifier             | 10-folds           | [100, 100, 100] |                            0.1421 |                              0.0728 | 0.0955                                    |
| tanh                  | None               | [5, 5, 5]       |                            0.2108 |                              0.2058 | -                                         |
| tanh                  | None               | [20, 20, 20]    |                            0.1627 |                              0.1271 | -                                         |
| tanh                  | None               | [100, 100, 100] |                            0.158  |                              0.0738 | -                                         |
| tanh                  | 5-folds            | [5, 5, 5]       |                            0.1978 |                              0.1778 | 0.2826                                    |
| tanh                  | 5-folds            | [20, 20, 20]    |                            0.1393 |                              0.138  | 0.2066                                    |
| tanh                  | 5-folds            | [100, 100, 100] |                            0.1355 |                              0.2241 | 0.1281                                    |
| tanh                  | 10-folds           | [5, 5, 5]       |                            0.1978 |                              0.1778 | 0.2587                                    |
| tanh                  | 10-folds           | [20, 20, 20]    |                            0.1393 |                              0.138  | 0.1837                                    |
| tanh                  | 10-folds           | [100, 100, 100] |                            0.1232 |                              0.1164 | 0.0929                                    |